In [4]:
import locale
locale.getpreferredencoding()

'cp936'

In [5]:
import newspaper
newspaper.languages()


Your available languages are:

input code		full name
  ar			  Arabic
  da			  Danish
  de			  German
  en			  English
  es			  Spanish
  fi			  Finnish
  fr			  French
  he			  Hebrew
  hu			  Hungarian
  id			  Indonesian
  it			  Italian
  ko			  Korean
  mk			  Macedonian
  nb			  Norwegian (Bokmål)
  nl			  Dutch
  no			  Norwegian
  pt			  Portuguese
  ru			  Russian
  sv			  Swedish
  tr			  Turkish
  vi			  Vietnamese
  zh			  Chinese



In [16]:
from newspaper import Article
url = 'http://news.sina.com.cn/c/nd/2016-06-16/doc-ifxtfrrc3682945.shtml'
url = 'http://news.sina.com.cn/zl/2016-06-16/doc-ifxtfrrf0436412.shtml'
a = Article(url, language='zh') # Chinese

# >>> a.download()
# >>> a.parse()

# >>> print(a.text[:150])
# 香港行政长官梁振英在各方压力下就其大宅的违章建
# 筑（僭建）问题到立法会接受质询，并向香港民众道歉。
# 梁振英在星期二（12月10日）的答问大会开始之际在其
# 演说中道歉，但强调他在违章建筑问题上没有隐瞒的意
# 图和动机。 一些亲北京阵营议员欢迎梁振英道歉，
# 且认为应能获得香港民众接受，但这些议员也质问梁振英有

# >>> print(a.title)
# 港特首梁振英就住宅违建事件道歉

In [17]:
a.download()
a.parse()


In [20]:
a.text
a.title
a.top_image

'http://i0.sinaimg.cn/dy/slidenews/41_img/2016_24/28161_5112892_128172.jpg'

![](http://i0.sinaimg.cn/dy/slidenews/41_img/2016_24/28161_5112892_128172.jpg)

In [21]:
from newspaper import Article
url = 'http://www.zhihu.com/question/47414054/answer/105880816'
# url = 'http://news.sina.com.cn/zl/2016-06-16/doc-ifxtfrrf0436412.shtml'
a = Article(url, language='zh') # Chinese
a.download()
a.parse()

In [33]:
print(a.title)
# print(a.text)
# print(a.html)
print(a.authors)
print(a.top_image)
print(a.movies)

如何看待 WWDC 2016 上推出的 Swift Playgrounds？
[]

[]


In [34]:
def h2t_handle_html(html):
  import html2text
  h2t = html2text.HTML2Text()
  h2t.body_width = 0
  return '\n'.join(p.rstrip() for p in h2t.handle(html).strip().split('\n'))


In [37]:
h2t_handle_html(a.html)


'[知乎](/)\n\n  * [__注册知乎](javascript:;)\n  * [登录](javascript:;)\n\n提问\n\n知乎搜索 搜索\n\n  * [首页](/)\n  * [话题](/topic)\n  * [发现](/explore)\n\n\n\n\n\n消息 用户 赞同和感谢\n\n[查看全部 »](/notifications) [__](/settings/notification "通知设置" )\n\n[x](javascript:; "关闭" )\n\n[ iOS ](/topic/19550461) [ 苹果公司 (Apple Inc.) ](/topic/19551762) [ Swift 语言 ](/topic/20003934) [ WWDC 2016 ](/topic/20046761)\n\n##  [如何看待 WWDC 2016 上推出的 Swift Playgrounds？](/question/47414054)\n\n__1 条评论  __分享\n\n### [查看全部 31 个回答](/question/47414054)\n\n__ 28 赞同 __ 反对\n\n[![](https://pic3.zhimg.com/2447e1332_s.jpg)](/people/huang-jing-cheng) [黄兢成](/people/huang-jing-cheng) ，iOS 和 C++ 开发者，[https://github.com/hjcapple__](//link.zhihu.com/?target=https%3A//github.com/hjcapple)\n\n收录于  编辑推荐 •28 人赞同\n\n整场发布会看下来， Swift Playgrounds 最有意思。以前我看过 Bret Victor 的一些视频。[Bret Victor - Inventing on Principle__](//link.zhihu.com/?target=http%3A//v.youku.com/v_show/id_XMzUyOTIyNzg0.html%3Ffrom%3Ds1.8-1-1.2)[Bret Victor - Media for Thinking The Unthinkable__](

In [1]:
from zhihu_oauth import ZhihuClient
TOKEN_FILE = 'token.pkl'
client = ZhihuClient()
client.load_token(TOKEN_FILE)

me = client.me()

print('name', me.name)
print('headline', me.headline)
print('description', me.description)

answer = client.answer(94150403)

print(answer.question.title)
print(answer.author.name)
print(answer.voteup_count)
print(answer.bthanks_count)
print(answer.created_time)
print(answer.updated_time)
answer.save(answer.question.title)

name Pobpob
headline 试试知乎有什么厉害的
description 
如何评价南开大学津南校区的建设质量？
7sDream
6


AttributeError: 'Answer' object has no attribute 'bthanks_count'

In [19]:
answer = client.answer(106335325)
comments = list(answer.comments)
for i, c in enumerate(comments, 1):
    print(i, c.content)

1 地主家也没有余粮啊
2 所以到最后便成了赚人民币花美元的锅咯
3 这实际上就是吸血效应
4 →_→一百多年前的事情重来
5 这个论点我记得马克思说过的，更高房价催生的是更庞大的产业后备军
6 让一部分人先买的起房。
7 玄处的文风……玄体
8 相比起马督工的洗地文，还是玄处的回答最好！
9 药丸
10 看不到武装革命可能性的情况下,读马列是不是会越读越郁闷?
11 无产阶级联合起来！
12 嗯，你个缓则，居然最后骂哮喘，人家容易吗？
13 可以用马列主义指导炒房嘛，郁闷啥呢？
14 早就说过，很多人以为2块钱的地铁是福利，那他妈的是为了让你住的更远。
15 活着真难
16 自从关注了玄处，督工的文章我已经懒得看了。
17 地铁还好啦，这点涨价翻不起多少浪花。可怕的是廉租房啊，这是用你交的税去盖房子，以便从外地再弄来一批屌丝和你抢饭碗。
18 还有火车票20年不涨价，很多人大呼良心，明显脑子缺根弦。
19 你觉得能问出这话的会是有钱炒房的阶级么...
20 意思是首都全国吸血，美国全世界吸血?
21 这个论点很有意思，感谢提供了一个新视角
22 如果天朝最终能够兵不血刃的征服（买下）整个美利坚，赵老爷家真是功在社稷，利在千秋啊
23 咋滴，准备闹革命？图样图森破。<br>无产阶级只有解放了全人类才能解放自己，各位以及各位的孙子都没希望。<br>丛林太危险，到草原就安全了。
24 地铁修到燕郊，意味着晚上燕郊族们只有多加班一个小时，才能把因为新地铁通车而新搬到三河、大厂来的外来务工人员PK下去。这句话有点疑问，问题是能不能PK掉别人是看你的水平，不是看你的加班时间，也不是看你的“工作时间+通勤时间”啊
25 玄处你个吃皇粮的处级干部说出这等话来，我替赵老爷和钱老爷们伤心呐，白养了
26 当我们谈论竞争对手的时候，指的是和同等水平的人竞争。意思是说，三河大厂忽然搬来一群和你水平相当的人，看你怎么破。
27 就拿手机来说，美国人负责操作系统以及关键CPU的设计生产掌握最核心的技术，日韩台负责次等配件的零件生产，中国负责组装，亚非拉提供原材料和市场。世界和谐~~
28 玄处什么都懂呀
29 问题在于，竞争不是“你们俩工作一个月，创造效益少的人滚蛋”（简称A），而是“你俩一起过来面试，水平高的留下，如果你创造的价值匹配了我给你发的钱，你就一直干下去”（简称B），如果竞争方式

In [11]:
for c in comments:
    print(c.author.name, ":", c.content)
    g = c.conversation
    print(g)

赵猫 : 地主家也没有余粮啊
夏辰 : 所以到最后便成了赚人民币花美元的锅咯
不想上吊王承恩 : 这实际上就是吸血效应
Kinetic AC : →_→一百多年前的事情重来
Chen Jian : 这个论点我记得马克思说过的，更高房价催生的是更庞大的产业后备军
谢祥祥 : 让一部分人先买的起房。
江瑜 : 玄处的文风……玄体
1781的小提琴 : 相比起马督工的洗地文，还是玄处的回答最好！
三两半 : 药丸
Razor Liu : 看不到武装革命可能性的情况下,读马列是不是会越读越郁闷?
哈哈哈同好会 : 无产阶级联合起来！
顾威 : 嗯，你个缓则，居然最后骂哮喘，人家容易吗？
玄不救非氪不改命 : 可以用马列主义指导炒房嘛，郁闷啥呢？
门中木 : 早就说过，很多人以为2块钱的地铁是福利，那他妈的是为了让你住的更远。
Lothlorien : 活着真难
罗晨光 : 自从关注了玄处，督工的文章我已经懒得看了。
玄不救非氪不改命 : 地铁还好啦，这点涨价翻不起多少浪花。可怕的是廉租房啊，这是用你交的税去盖房子，以便从外地再弄来一批屌丝和你抢饭碗。
门中木 : 还有火车票20年不涨价，很多人大呼良心，明显脑子缺根弦。
Razor Liu : 你觉得能问出这话的会是有钱炒房的阶级么...
郁青青 : 意思是首都全国吸血，美国全世界吸血?
刘书宁 : 这个论点很有意思，感谢提供了一个新视角
杨丹尘 : 如果天朝最终能够兵不血刃的征服（买下）整个美利坚，赵老爷家真是功在社稷，利在千秋啊
马在野 : 咋滴，准备闹革命？图样图森破。<br>无产阶级只有解放了全人类才能解放自己，各位以及各位的孙子都没希望。<br>丛林太危险，到草原就安全了。
刘书宁 : 地铁修到燕郊，意味着晚上燕郊族们只有多加班一个小时，才能把因为新地铁通车而新搬到三河、大厂来的外来务工人员PK下去。这句话有点疑问，问题是能不能PK掉别人是看你的水平，不是看你的加班时间，也不是看你的“工作时间+通勤时间”啊
林荣华 : 玄处你个吃皇粮的处级干部说出这等话来，我替赵老爷和钱老爷们伤心呐，白养了
玄不救非氪不改命 : 当我们谈论竞争对手的时候，指的是和同等水平的人竞争。意思是说，三河大厂忽然搬来一群和你水平相当的人，看你怎么破。
林荣华 : 就拿手机来说，美国人负责操作系统以及关键CPU的设计生产掌握最核心的技术，日韩

In [32]:
c0 = comments[0]
c12 = comments[12]
print(c12.content)
g= c0.conversation
print(g)
replies = list(c0.replies)
[print(r.content) for r in replies]
print()


replies12 = list(c12.replies) # 所有回复本评论的评论, 第一条为本评论
[print(r.author.name, r.content) for r in replies12]
print()


g12 = list(c12.conversation)  # 包含该评论的对话, 从最开始到结束
[print(r.author.name, r.content) for r in g12]

可以用马列主义指导炒房嘛，郁闷啥呢？
地主家也没有余粮啊

玄不救非氪不改命 可以用马列主义指导炒房嘛，郁闷啥呢？
Razor Liu 你觉得能问出这话的会是有钱炒房的阶级么...
暗黑的傀儡师 思路很新颖，就是把劳动力市场看的太简单了，不是还有简单劳动和复杂劳动之分，住燕郊的人大部分也不是富士康那种厂工，所以工作时间和产出之间并不是线性的；另外，老板给租了房，员工很可能更不愿意加班了。

Razor Liu 看不到武装革命可能性的情况下,读马列是不是会越读越郁闷?
玄不救非氪不改命 可以用马列主义指导炒房嘛，郁闷啥呢？
Razor Liu 你觉得能问出这话的会是有钱炒房的阶级么...


[None, None, None]